In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [11]:
# Создаём сэмпл: лаботаторные отделения, в которых ведут прием врачи по специализациям sp1, sp2, sp3
n_samples = 500

# расстояние от лабораторного отделения до ближайшего метро
dist_to_metro = np.random.choice(5, n_samples)
# количество лицензий лабораторного отделения
emp = np.random.randint(0, 10, n_samples) + 1
# кабинеты врачей 1,2,3
sp1 = np.random.randint(1, 2, n_samples)
sp2 = np.random.randint(0, 2, n_samples)
sp3 = np.random.randint(0, 2, n_samples)
# сколько дней уже открыто отделение
working_days = np.random.choice(3660, n_samples)
# среднее количество посетителей в день
visitors = 5 *((sp1 + sp2 + sp3)*(sp1 + sp2 + sp3)) - 0.01 * dist_to_metro + 0.05 * working_days
av_bill = 1300 + np.random.randint(-500, 500, n_samples)
# средняя дневная выручка
earnings = visitors * av_bill 

data = pd.DataFrame({'dist_to_metro': dist_to_metro, 'working_days': working_days, 'av_bill': av_bill, 'earnings': earnings, 'emp': emp, 'sp1': sp1, 'sp2': sp2,'sp3': sp3, 'visitors': visitors})
data.head(5)


,dist_to_metro,working_days,av_bill,earnings,emp,sp1,sp2,sp3,visitors
0,2,1115,1365,82896.45,9,1,0,0,60.73
1,1,3514,1141,223282.29,10,1,0,1,195.69
2,1,456,1124,76195.96,1,1,1,1,67.79
3,3,1849,1604,180321.68,10,1,1,0,112.42
4,0,631,1411,72737.05,6,1,0,1,51.55


строю линейную регрессию для прогнозирования выручки earnings 

In [12]:
X = data[['dist_to_metro', 'working_days', 'visitors','emp', 'sp1', 'sp2', 'sp3']]
y = data['earnings']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['dist_to_metro', 'working_days', 'visitors', 'emp', 'sp1', 'sp2', 'sp3']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 6.77686197e+02  6.90655423e+01 -9.34440249e+01  3.78995628e+02
  4.72937245e-11  2.04105526e+04  2.73034286e+04]
Bias: 4325.809566980577
Error: 28443.18929942751


Ошибка 28443 рубля, что вероятно составляет порядка 15% от суммы выручки.
Наш генеральный директор уволил бы за такой прогноз))

Создам новый признак "Количество специалистов в отделении".
Возможно каждый врач в отдельности не влияет на выручку и важно только количество кабинетов, открытых в лабораторном отделении


In [13]:
data['all_sp'] = data['sp1'] + data['sp2'] + data['sp3']
data.head()

,dist_to_metro,working_days,av_bill,earnings,emp,sp1,sp2,sp3,visitors,all_sp
0,2,1115,1365,82896.45,9,1,0,0,60.73,1
1,1,3514,1141,223282.29,10,1,0,1,195.69,2
2,1,456,1124,76195.96,1,1,1,1,67.79,3
3,3,1849,1604,180321.68,10,1,1,0,112.42,2
4,0,631,1411,72737.05,6,1,0,1,51.55,2


In [14]:
X = data[['dist_to_metro', 'working_days','emp', 'visitors', 'all_sp']]
y = data['earnings']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['dist_to_metro', 'working_days','emp', 'visitors', 'all_sp']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [  613.72309078    66.22091345   413.74253994   -36.01320008
 22709.36065294]
Bias: -18781.11346349228
Error: 28469.900635583752


ошибка незначительно, но стала больше

считаю, что количество лицензий отделения emp не должно влиять на выручку, убираю эту переменную


In [15]:
X = data[['dist_to_metro', 'working_days', 'visitors', 'sp1', 'sp2', 'sp3']]
y = data['earnings']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['dist_to_metro', 'working_days', 'visitors', 'sp1', 'sp2', 'sp3']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 6.95046630e+02  6.99877965e+01 -1.13253135e+02  1.78260962e-10
  2.07497711e+04  2.77693141e+04]
Bias: 6580.271376037388
Error: 28436.710019714163


ошибка стала незначительно больше

хочу посмотреть модель без учета "расстояния до метро" и "дней работы отделения"

In [16]:
X = data[['visitors', 'sp1', 'sp2', 'sp3']]
y = data['earnings']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['visitors', 'sp1', 'sp2', 'sp3']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 1282.6925243      0.         -6690.34867834  -346.54628794]
Bias: 4651.1699571695935
Error: 28651.555430485078


ошибка увеличилась на 200 руб. 
убираю все переменные кроме "количество посетителей"

In [17]:
X = data[['visitors']]
y = data['earnings']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['visitors']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [1271.98830713]
Bias: 2272.930584553367
Error: 28634.548391652857


результат стал хуже

Самая удачная модель получается, если убрать переменную "количество лицензий" и оставить все остальные переменные.
Но в целом можно строить прогнозы только по количеству посетителей, так как разница между ошибкой в этой модели и 
ошибкой в самой удачной модели небольшая.